<a href="https://colab.research.google.com/github/karlychann/neuro140/blob/main/uvfp_testrf_one_dataset_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final models trained on the entire dataset, with and without biased features

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from io import BytesIO
import pickle
import requests
pd.set_option('display.max_columns', None)

In [ ]:
all_features = ['F0semitoneFrom27.5Hz_sma3nz_amean',
       'F0semitoneFrom27.5Hz_sma3nz_stddevNorm',
       'F0semitoneFrom27.5Hz_sma3nz_percentile20.0',
       'F0semitoneFrom27.5Hz_sma3nz_percentile50.0',
       'F0semitoneFrom27.5Hz_sma3nz_percentile80.0',
       'F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2',
       'F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope',
       'F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope',
       'F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope',
       'F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope', 'loudness_sma3_amean',
       'loudness_sma3_stddevNorm', 'loudness_sma3_percentile20.0',
       'loudness_sma3_percentile50.0', 'loudness_sma3_percentile80.0',
       'loudness_sma3_pctlrange0-2', 'loudness_sma3_meanRisingSlope',
       'loudness_sma3_stddevRisingSlope', 'loudness_sma3_meanFallingSlope',
       'loudness_sma3_stddevFallingSlope', 'spectralFlux_sma3_amean',
       'spectralFlux_sma3_stddevNorm', 'mfcc1_sma3_amean',
       'mfcc1_sma3_stddevNorm', 'mfcc2_sma3_amean', 'mfcc2_sma3_stddevNorm',
       'mfcc3_sma3_amean', 'mfcc3_sma3_stddevNorm', 'mfcc4_sma3_amean',
       'mfcc4_sma3_stddevNorm', 'jitterLocal_sma3nz_amean',
       'jitterLocal_sma3nz_stddevNorm', 'shimmerLocaldB_sma3nz_amean',
       'shimmerLocaldB_sma3nz_stddevNorm', 'HNRdBACF_sma3nz_amean',
       'HNRdBACF_sma3nz_stddevNorm', 'logRelF0-H1-H2_sma3nz_amean',
       'logRelF0-H1-H2_sma3nz_stddevNorm', 'logRelF0-H1-A3_sma3nz_amean',
       'logRelF0-H1-A3_sma3nz_stddevNorm', 'F1frequency_sma3nz_amean',
       'F1frequency_sma3nz_stddevNorm', 'F1bandwidth_sma3nz_amean',
       'F1bandwidth_sma3nz_stddevNorm', 'F1amplitudeLogRelF0_sma3nz_amean',
       'F1amplitudeLogRelF0_sma3nz_stddevNorm', 'F2frequency_sma3nz_amean',
       'F2frequency_sma3nz_stddevNorm', 'F2bandwidth_sma3nz_amean',
       'F2bandwidth_sma3nz_stddevNorm', 'F2amplitudeLogRelF0_sma3nz_amean',
       'F2amplitudeLogRelF0_sma3nz_stddevNorm', 'F3frequency_sma3nz_amean',
       'F3frequency_sma3nz_stddevNorm', 'F3bandwidth_sma3nz_amean',
       'F3bandwidth_sma3nz_stddevNorm', 'F3amplitudeLogRelF0_sma3nz_amean',
       'F3amplitudeLogRelF0_sma3nz_stddevNorm', 'alphaRatioV_sma3nz_amean',
       'alphaRatioV_sma3nz_stddevNorm', 'hammarbergIndexV_sma3nz_amean',
       'hammarbergIndexV_sma3nz_stddevNorm', 'slopeV0-500_sma3nz_amean',
       'slopeV0-500_sma3nz_stddevNorm', 'slopeV500-1500_sma3nz_amean',
       'slopeV500-1500_sma3nz_stddevNorm', 'spectralFluxV_sma3nz_amean',
       'spectralFluxV_sma3nz_stddevNorm', 'mfcc1V_sma3nz_amean',
       'mfcc1V_sma3nz_stddevNorm', 'mfcc2V_sma3nz_amean',
       'mfcc2V_sma3nz_stddevNorm', 'mfcc3V_sma3nz_amean',
       'mfcc3V_sma3nz_stddevNorm', 'mfcc4V_sma3nz_amean',
       'mfcc4V_sma3nz_stddevNorm', 'alphaRatioUV_sma3nz_amean',
       'hammarbergIndexUV_sma3nz_amean', 'slopeUV0-500_sma3nz_amean',
       'slopeUV500-1500_sma3nz_amean', 'spectralFluxUV_sma3nz_amean',
       'loudnessPeaksPerSec', 'VoicedSegmentsPerSec',
       'MeanVoicedSegmentLengthSec', 'StddevVoicedSegmentLengthSec',
       'MeanUnvoicedSegmentLength', 'StddevUnvoicedSegmentLength',
       'equivalentSoundLevel_dBp']


# from classification_wo_correlated_features.ipynb features that correlate least with biased features
uncorrelated_features = ['mfcc4V_sma3nz_amean',
       'F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope', 'mfcc1_sma3_amean',
       'F3bandwidth_sma3nz_stddevNorm',
       'F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope',
       'F1frequency_sma3nz_stddevNorm', 'jitterLocal_sma3nz_stddevNorm',
       'F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope',
       'alphaRatioV_sma3nz_stddevNorm', 'mfcc1_sma3_stddevNorm',
       'F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope', 'mfcc4_sma3_amean',
       'F3frequency_sma3nz_amean', 'mfcc2_sma3_amean',
       'VoicedSegmentsPerSec', 'F1bandwidth_sma3nz_amean',
       'mfcc2V_sma3nz_amean', 'F3frequency_sma3nz_stddevNorm',
       'hammarbergIndexV_sma3nz_stddevNorm',
       'logRelF0-H1-H2_sma3nz_amean', 'slopeV500-1500_sma3nz_stddevNorm',
       'F2bandwidth_sma3nz_amean', 'mfcc3_sma3_amean',
       'F2bandwidth_sma3nz_stddevNorm', 'alphaRatioV_sma3nz_amean',
       'mfcc2_sma3_stddevNorm', 'mfcc1V_sma3nz_amean',
       'slopeUV0-500_sma3nz_amean', 'mfcc1V_sma3nz_stddevNorm',
       'mfcc3V_sma3nz_amean', 'F2frequency_sma3nz_amean',
       'logRelF0-H1-A3_sma3nz_amean', 'hammarbergIndexV_sma3nz_amean',
       'F1bandwidth_sma3nz_stddevNorm', 'mfcc3_sma3_stddevNorm',
       'mfcc2V_sma3nz_stddevNorm', 'F1frequency_sma3nz_amean',
       'F2frequency_sma3nz_stddevNorm',
       'logRelF0-H1-H2_sma3nz_stddevNorm', 'mfcc4V_sma3nz_stddevNorm',
       'mfcc4_sma3_stddevNorm', 'F3bandwidth_sma3nz_amean',
       'mfcc3V_sma3nz_stddevNorm', 'slopeV0-500_sma3nz_stddevNorm']

In [ ]:
# load pretrained model
# model_name = 'less-biased'
# training_model_name = 'rf'
# task = 'speech'
# feature_set = uncorrelated_features

# url_path = f'https://github.com/danielmlow/vfp/blob/main/data/output/{training_model_name}_{model_name}_{task}.pkl?raw=true' # speech models trained on reading task
# mfile = BytesIO(requests.get(url_path).content) # load from url
# model = pickle.load(mfile)

# Extract features on your own wav files using egemaps


To test on your own data, the test set should match our features (egemaps) using the the same variables and sampling rate (16k)

In [ ]:
# from os.path import exists
# # config: depends whether you're on Google Colab or local


# # Get URL from github csv by clicking on Download > Copy Link Address

# load_from_google_drive = False

# if load_from_google_drive:
#       # On google colab
#       # Mount GDrive and attach it to the colab for data I/O
#     from google.colab import drive
#     drive.mount('/content/drive')
#     input_dir = '/content/drive/My Drive/datum/vfp/data/input/'
#     output_dir = '/content/drive/My Drive/datum/vfp/data/output/'
#     os.makedirs(output_dir, exist_ok=True)

# else:
#   # If using jupyter-lab or jupyter notebook, load locally:
#   input_dir = './data/input/'
#   output_dir = './data/output/'



In [ ]:
from google.colab import drive
drive.flush_and_unmount()

from google.colab import drive
drive.mount('/content/drive')
input_dir = '/content/drive/My Drive/neuro140/vfp/data/input/'
output_dir = '/content/drive/My Drive/neuro140/vfp/data/output/'
os.makedirs(output_dir, exist_ok=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
# !unzip /content/drive/MyDrive/neuro140/vfp/data/archive.zip -d /content/drive/MyDrive/neuro140/vfp/data

In [ ]:
! grep 'Vocal fold paralysis' /content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/*.txt

/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice087-info.txt:Diagnosis:	hypokinetic dysphonia (Vocal fold paralysis)
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice093-info.txt:Diagnosis:	hypokinetic dysphonia (Vocal fold paralysis)
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice112-info.txt:Diagnosis:	hypokinetic dysphonia (Vocal fold paralysis)
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice136-info.txt:Diagnosis:	hypokinetic dysphonia (Vocal fold paralysis)
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice170-info.txt:Diagnosis:	hyperkinetic dysphonia (Vocal fold paralysis)


In [ ]:
# ! grep 'healthy' /content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/*.txt | head

/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice002-info.txt:Diagnosis:	healthy
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice019-info.txt:Diagnosis:	healthy
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice024-info.txt:Diagnosis:	healthy
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice025-info.txt:Diagnosis:	healthy
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice032-info.txt:Diagnosis:	healthy
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice034-info.txt:Diagnosis:	healthy
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice040-info.txt:Diagnosis:	healthy
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice045-info.txt:Diagnosis:	healthy
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice049-info.txt:Diagnosis:	healthy
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice051-info.txt:Diagnosis:	healthy


In [ ]:
#pip install wfdb scipy

In [ ]:
# import wfdb
# from scipy.io.wavfile import write

# def wfdb_to_wav(input_path, output_path, channel=0):
#     """
#     Convert a WFDB file to a WAV file.

#     Parameters:
#     - input_path: Path to the input WFDB file.
#     - output_path: Path to the output WAV file.
#     - channel: The channel of the WFDB file to convert (default is 0)
#     """
#     # Read the WFDB file
#     record = wfdb.rdrecord(input_path)

#     # Extract the signal from the specified channel
#     signal = record.p_signal[:, channel]

#     # Normalize the signal to be in the range of int16 (required for WAV files)
#     signal_normalized = ((signal - signal.min()) / (signal.max() - signal.min()) * (2**15 - 1) - 2**15).astype('int16')

#     # Write the normalized signal to a WAV file
#     # Note: The sample rate is set according to the WFDB record's sampling frequency
#     write(output_path, record.fs, signal_normalized)


# new_data = ['voice087', 'voice093', 'voice112', 'voice136', 'voice170',
#             'voice002', 'voice019', 'voice024', 'voice025', 'voice032']
# for input in new_data:
#   wfdb_to_wav(f'/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/{input}'
#               , f'/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/{input}.wav')


In [ ]:
# ! rm /content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice001.wav


In [ ]:
! ls /content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/*.wav

/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice002.wav
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice019.wav
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice024.wav
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice025.wav
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice032.wav
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice087.wav
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice093.wav
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice112.wav
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice136.wav
/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/voice170.wav


In [ ]:
# !pip install -q opensmile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.2/996.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# import glob
# import opensmile

In [ ]:
# wav_dir = '/content/drive/MyDrive/neuro140/vfp/data/VOICEDDATASET/*.wav'
# wav_paths = glob.glob(wav_dir)

In [ ]:
# smile = opensmile.Smile(
#             feature_set=opensmile.FeatureSet.eGeMAPSv02, #or path to conf: 'gemaps/eGeMAPSv02.conf'
#             feature_level=opensmile.FeatureLevel.Functionals,
#             sampling_rate=16000,
#             resample=True,
#             # num_workers = 4,
#             verbose=True,
#         )
# feature_vectors = smile.process_files(wav_paths)
# df_voiced = feature_vectors.reset_index()


In [ ]:
! find '/content/drive/MyDrive/neuro140/vfp/data/' -name '*.csv**'

/content/drive/MyDrive/neuro140/vfp/data/extracted_voiced.csv


In [ ]:
df_voiced = pd.read_csv('/content/drive/MyDrive/neuro140/vfp/data/extracted_voiced.csv')


# Training these models

In [ ]:
# We choose Random Forest as it tends to have highest median classification across analyses
# model = RandomForestClassifier(n_estimators= 100)

# Others:
# LogisticRegressionCV(solver='liblinear', penalty = 'l1', max_iter = 100)
# MLPClassifier(alpha = 1, max_iter= 1000)
# SGDClassifier(loss='log', penalty="elasticnet", early_stopping=True, max_iter = 5000)

# Random Forest Model

In [ ]:
training_model_name = 'rf'

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score
import pandas as pd
import pickle

# Define a random seed for reproducibility
random_seed = 38

# Assume df_voiced, all_features, uncorrelated_features, and output_dir are defined elsewhere

for task in ['speech', 'vowel']:
    df = pd.read_csv(f'https://github.com/danielmlow/vfp/raw/main/data/input/features/egemaps_vector_{task}_cpp.csv', index_col=0)

    for feature_set, model_name in zip([all_features, uncorrelated_features],  # Assuming these are defined
                                       ['biased', 'less-biased']):

        X = df[feature_set].values
        y = df['target'].values

        # Define the model
        model = RandomForestClassifier(n_estimators=100, random_state=random_seed)

        # Initialize StratifiedKFold to maintain the target distribution within each fold
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)

        # Perform 5-fold cross-validation and compute the ROC AUC score for each fold
        # Note: It's important to use a metric suitable for your problem, here we assume binary classification
        cv_scores = cross_val_score(model, X, y, cv=cv, scoring='roc_auc')

        # Print out the mean cross-validation score
        print(f'5-Fold CV ROC AUC Scores for {task} - {model_name} - {training_model_name}:', cv_scores)
        print(f'Mean CV ROC AUC for {task} - {model_name} - {training_model_name} :', cv_scores.mean())

        # You might still want to train and save the final model on the entire dataset
        model.fit(X, y)  # Train the model on the entire dataset

        X_test = df_voiced[feature_set].values
        y_test = df_voiced['target'].values

        # Predict probabilities for the test data
        y_probs = model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

        # Calculate ROC AUC for each class and take the average
        roc_auc = roc_auc_score(y_test, y_probs)
        print(f'ROC AUC Score: {roc_auc}')

        # Save the trained model
        output_path = f'{output_dir}{task}_{model_name}_{training_model_name}_noVOICED.pkl'  # Assuming output_dir is defined
        with open(output_path, 'wb') as f:
            pickle.dump(model, f)

        # Optional: Load model example (commented)
        # with open(output_path, 'rb') as f:
        #     model = pickle.load(f)

        # Since we're now using cross-validation, the performance metric printed here would be from CV
        print(f'Completed training and saving for {task} - {model_name} - {training_model_name}with mean CV ROC AUC: {cv_scores.mean()}')

5-Fold CV ROC AUC Scores for speech - biased - rf: [0.96835749 0.90072464 0.95647969 0.95701581 0.96195652]
Mean CV ROC AUC for speech - biased - rf : 0.9489068296283838
ROC AUC Score: 0.04000000000000001
Completed training and saving for speech - biased - rfwith mean CV ROC AUC: 0.9489068296283838
5-Fold CV ROC AUC Scores for speech - less-biased - rf: [0.92801932 0.89202899 0.9115087  0.90909091 0.92613636]
Mean CV ROC AUC for speech - less-biased - rf : 0.9133568571935824
ROC AUC Score: 0.9400000000000001
Completed training and saving for speech - less-biased - rfwith mean CV ROC AUC: 0.9133568571935824
5-Fold CV ROC AUC Scores for vowel - biased - rf: [0.99227053 0.95821256 0.97294686 0.95797101 0.94801741]
Mean CV ROC AUC for vowel - biased - rf : 0.9658836748614732
ROC AUC Score: 0.8400000000000001
Completed training and saving for vowel - biased - rfwith mean CV ROC AUC: 0.9658836748614732
5-Fold CV ROC AUC Scores for vowel - less-biased - rf: [0.97294686 0.93743961 0.93719807 0

In [ ]:
y_probs = model.predict_proba(X_test) # Get probabilities for the positive class
y_predict = model.predict(X_test)
y_probs[: ,0], y_predict
y_test

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

# Tested to hyperparameter tune using the below code and save into file

In [ ]:
training_model_name = 'rf'

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import pandas as pd
import pickle

# Define a random seed for reproducibility
random_seed = 42

new_param_grids = {
    ('speech', 'biased') : {
      'n_estimators': [300],
      'max_depth': [None],
      'min_samples_split': [5],
      'min_samples_leaf': [2],
      'max_features': ['auto'],
      'bootstrap': [False]
    },
    ('speech', 'less-biased') : {
      'n_estimators': [200],
      'max_depth': [None],
      'min_samples_split': [2],
      'min_samples_leaf': [2],
      'max_features': ['sqrt'],
      'bootstrap': [True]
    },
    ('vowel', 'biased') : {
      'n_estimators': [100],
      'max_depth': [10],
      'min_samples_split': [2],
      'min_samples_leaf': [1],
      'max_features': ['auto'],
      'bootstrap': [True]
    },
    ('vowel', 'less-biased') : {
      'n_estimators': [200],
      'max_depth': [30],
      'min_samples_split': [2],
      'min_samples_leaf': [1],
      'max_features': ['sqrt'],
      'bootstrap': [False]
    },
}


# Initialize StratifiedKFold
cv = StratifiedKFold(n_splits=5)

for task in ['speech', 'vowel']:
    df = pd.read_csv(f'https://github.com/danielmlow/vfp/raw/main/data/input/features/egemaps_vector_{task}_cpp.csv', index_col=0)

    for feature_set, model_name in zip([all_features, uncorrelated_features],  # Assuming these are defined
                                       ['biased', 'less-biased']):

        # train with Low dataset
        X = df[feature_set].values
        y = df['target'].values

        # Initialize the classifier
        model = RandomForestClassifier(n_estimators=100, random_state=random_seed)

        # Initialize StratifiedKFold
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)

        # Set up GridSearchCV
        param_grid = new_param_grids[(task, model_name)]
        grid_search = GridSearchCV(model, param_grid, scoring='roc_auc', cv=cv, verbose=1, n_jobs=-1)
        grid_search.fit(X, y)

        # Best model after grid search
        best_model = grid_search.best_estimator_

        # Print out the best parameters and mean cross-validation score
        print(f'Best parameters for {task} - {model_name}:', grid_search.best_params_)
        print(f'Best CV score for {task} - {model_name}:', grid_search.best_score_)

        # Save the trained model
        output_path = f'{output_dir}{task}_{model_name}_{training_model_name}_hypertuned2_NoVOICED.pkl'  # Assuming output_dir is defined
        with open(output_path, 'wb') as f:
            pickle.dump(best_model, f)

        # test with VOICED dataset
        X_test = df_voiced[feature_set].values
        y_test = df_voiced['target'].values

        # Predict probabilities for the test data
        y_probs = best_model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

        # Calculate ROC AUC for each class and take the average
        roc_auc = roc_auc_score(y_test, y_probs)
        print(f'Low Train, ROC AUC Score: {roc_auc}')

        # Save the best parameters in a file
        output_path = f'{output_dir}{task}_{model_name}_{training_model_name}_hypertunedparameters2_NoVOICED.pkl'  # Assuming output_dir is defined
        with open(output_path, 'wb') as f:
            pickle.dump(grid_search.best_params_, f)

        print(f'Model saved for {task} - {model_name} with parameters {grid_search.best_params_}')


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best parameters for speech - biased: {'bootstrap': False, 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}
Best CV score for speech - biased: 0.9540939926555098
Low Train, ROC AUC Score: 0.32
Model saved for speech - biased with parameters {'bootstrap': False, 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters for speech - less-biased: {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Best CV score for speech - less-biased: 0.9107861220904698
Low Train, ROC AUC Score: 0.5800000000000001
Model saved for speech - less-biased with parameters {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Fitting 5 folds for each of 1 candidates, tota

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best parameters for vowel - biased: {'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best CV score for vowel - biased: 0.961635597417281
Low Train, ROC AUC Score: 0.6799999999999999
Model saved for vowel - biased with parameters {'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters for vowel - less-biased: {'bootstrap': False, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best CV score for vowel - less-biased: 0.9436122557676674
Low Train, ROC AUC Score: 0.5
Model saved for vowel - less-biased with parameters {'bootstrap': False, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


# Other features

In [ ]:
cpp_features = ['cpp_amean', 'cpp_stddevNorm', 'cpp_percentile20', 'cpp_percentile80']